## Image segmentation with CamVid

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *

Untar the camvid data.

Set two paths: one to the labels at `path_lbl` and one to the images at `path_img`.

## Data

Get the image files using `get_image_files`. Print the first three.

Get the label image files also using `get_image_files`. Print the first three.

Show the first image.

Observe the image names and labels. Write a function that converts a given image name into its corresponding label name.

Use `open_mask` on the mask image name, and show it with the `show` method.

Create a variable called `src_size` with the length and width values of the mask. Print that along with actual mask data values.

Use `np.loadtxt` to load the text values from `path/'codes.txt'`.

## Datasets

Set size to `src_size//2`. Set `bs = 4`. 

Create a `SegmentationItemList` from folder `path_img`, split by fname file `../valid.txt`, label from func `get_y_fn` with `classes=codes`.

Turn that into a databunch with standard transforms, `size=size`, `tfm_y=True`, `bs=bs`, and normalize using imagenet stats.

Show a 2x2 batch with figsize 10,7 on the training set.

Same but validation set.

## Model

Create a dict `name2id` that maps an enumerated index to each code in the `codes` variable. Bind the variable `void_code` to `name2id['Void']`.

Create a function `acc_camvid` that takes an input and a target and gets the proportion of pixels where:
* The prediction from `input` matches the target
* The target pixel's code is not equal to `void_code`

Bind `acc_camvid` to the variable `metrics`.

Bind the value `1e-2` to the variable `wd` (for weight decay).

Create a `unet_learner` based on the resnet 34 architecture, using the `metrics` and `wd` variables created above.

Run lr_find to determine the appropriate learning raate and plot it.

The lr should be around `3e-3`.

Fit a cycle with 10 epochs with `slice(lr)`, `pct_start=0.9`.

Plot the losses.

Save the weights to `stage-1`.

Load the weights from `stage-1`.

Show 3 rows of results with figsize (8,9).

Unfreeze the model.

Set a slice of lrs from `lr/400` to `lr/4`.

Fit a cycle with 12 epochs using the new lrs, with `pct_start=0.8`.

Save these weights to `stage-2`.

## Go big

Set the batch size to 1 (you can try 1-3 if `gpu_mem_get_free_no_cache()` is high enough -- the notebooks suggest using 3 if it's > 8200). 

Create a databunch from `src` using standard transforms, the new size, transforms on y, batch size equal to whatever is set above, normalized by imagenet stats. Bind it to the name `data`. 

Create a unet learner from `data` using architecture resnet34 with the same metrics and wd as earlier.

Load the weights from `stage-2` above.

Use the learning rate finder.

You should have landed around `1e-3`. Bind the value you found to the `lr` variable.

Fit a cycle w/ 10 epochs, using a plain `slice` for lr, with `pct_start=0.8`.

Save the result to `stage-1-big`.

Load the result from `stage-1-big`.

Unfreeze the learner.

Create a new slice of `lrs` going from `1e-6` to `lr/10`.

Fit another cycle of 10 epochs with these lrs.

Save the weights to `stage-2-big`.

Load the weights from `stage-2-big`.

Show 3 rows of results with figsize 10,10.